<a href="https://colab.research.google.com/github/jrojasquiroz/hppAI/blob/main/evacal/carpetas_evacal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os
import pandas as pd
import shutil
from PIL import Image
import random

Con el siguiente código creo carpetas tal y como lo he pensado: una por cada columna, y dentro de ellas `train`, `test` y `validation`, tal como en [el ejemplo](https://www.youtube.com/watch?v=KZwuhl4hXok&t=83s) que revisado.

En este caso, las creo dentro de `orden_tercer_intento`.

In [51]:
# Ruta base
ruta_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento"

# Lista de carpetas y subcarpetas
carpetas = {
    "C_contempo": ["train", "test", "validation"],
    "C_estado": ["train", "test", "validation"],
    "C_armarios": ["train", "test", "validation"],
    "B_contempo": ["train", "test", "validation"],
    "B_estado": ["train", "test", "validation"],
    "B_lavamano": ["train", "test", "validation"],
    "R_contempo": ["train", "test", "validation"],
    "R_estado": ["train", "test", "validation"],
    "R_carpinteria": ["train", "test", "validation"],
    "R_singular": ["train", "test", "validation"],
    "R_ventana": ["train", "test", "validation"]
}

# Crear las carpetas y subcarpetas
for carpeta, subcarpetas in carpetas.items():
    carpeta_path = os.path.join(ruta_base, carpeta)
    os.makedirs(carpeta_path, exist_ok=True)
    for subcarpeta in subcarpetas:
        subcarpeta_path = os.path.join(carpeta_path, subcarpeta)
        os.makedirs(subcarpeta_path, exist_ok=True)
        # Crear subcarpetas adicionales dentro de train, test y validation
        if carpeta == "C_estado" or carpeta == "B_estado" or carpeta == "R_estado":
            for i in range(3):
                subcarpeta_adicional_path = os.path.join(subcarpeta_path, str(i))
                os.makedirs(subcarpeta_adicional_path, exist_ok=True)
        else:
            subcarpeta_adicional_path = os.path.join(subcarpeta_path, "0")
            os.makedirs(subcarpeta_adicional_path, exist_ok=True)
            subcarpeta_adicional_path = os.path.join(subcarpeta_path, "1")
            os.makedirs(subcarpeta_adicional_path, exist_ok=True)

Ahora voy a limpiar el dataframe que tiene la evaluación de cada caso.

In [43]:
df=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/tercer_intento/evaluacion_inicio.xlsx')
print(df.columns.tolist())

['ID_general', 'ID', 'Title', 'Link', 'precio_euros', 'codigo_inmueble', 'Description', 'last_mod', 'dummy_indice', 'indice_alquiler', 'area', 'n_rooms', 'price_m2', 'n_baths', 'n_aseo', 'laundry', 'buhardilla', 'despacho', 'trastero', 'terrace', 'kwhm2_year', 'kgCO2m2_year', 'dummy_consum_EPC', 'dummy_emission_EPC', 'consum_A', 'consum_B', 'consum_C', 'consum_D', 'consum_E', 'consum_F', 'consum_G', 'emission_A', 'emission_B', 'emission_C', 'emission_D', 'emission_E', 'emission_F', 'emission_G', 'AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'barrio', 'codigo_inmueble_numero', 'Lat_Y', 'Lon_X', 'multifam', 'atico', 'duplex', 'estudio', 'loft', 'piso', 'triplex', 'AMB', 'MAD', 'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido', 'Responsable']


In [44]:
df= df[df['Anuncio_invalido'].isna()]

In [45]:
df=df[['ID_general','C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']]

In [46]:
len(df)

86

In [47]:
df['R_singular'].isna().sum()

0

In [48]:
df[['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']]=df[['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
       'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']].astype(int)

Hay que tener cuidado que `df` siempre tenga esta apariencia. Si, por ejemplo, en lugar de valores enteros hubieran decimales, la operación hacia adelante puede arrojar errores.

Otra de las situaciones que observé fue que habían algunas columnas habían sido dejadas en blanco por error durante el llenado de datos.

In [49]:
df.head()

,ID_general,C_contempo,C_estado,C_armarios,B_contempo,B_estado,B_lavamano,R_contempo,R_estado,R_carpinteria,R_singular,R_ventana
3,4,1,2,1,1,2,1,0,1,1,0,1
5,6,1,2,1,1,2,1,1,2,1,0,0
6,7,1,2,1,1,2,1,1,2,1,0,1
11,12,1,2,1,1,2,1,1,2,1,0,1
12,13,1,2,1,1,2,1,1,2,2,0,1


A continuación, dado que en `tercer_intento` es en donde están almacenadas las imágenes tal y como los chicos las descargarán, reviso esa carpeta y correspondientes sub-carpetas para trasladarlas hacia `orden_tercer_intento`.

He preferido hacerlo con Google Colab porque al hacerlo directamente con Jupyter Notebook recibía el mensaje de que no tenía permiso de administrador del sistema (esto en la PC del CPSV, no lo probé en la laptop).

In [52]:
# Ruta del archivo Excel
#excel_path = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/tercer_intento/evaluacion_inicio.xlsx"

# Ruta base para las imágenes
imagenes_base = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/tercer_intento/imagenes"

# Ruta base para las imágenes modificadas
imagenes_mod = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento"

# Leer el archivo Excel
#df = pd.read_excel(excel_path)

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    caso_id = str(row['ID_general'])  # Obtener el ID del caso

    # Iterar sobre las letras (B, C, R)
    for letra in ['B', 'C', 'R']:
        carpeta_caso = os.path.join(imagenes_base, caso_id, letra)

        # Iterar sobre las columnas relevantes
        for columna in ['C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano',
                        'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana']:
            valor = row[columna]
            # Determinar la ruta de destino según la calificación
            ruta_destino = os.path.join(imagenes_mod, columna, 'train', str(valor))

            # Copiar la imagen a la carpeta de destino
            for item in os.listdir(carpeta_caso):
                # Obtener la ruta completa del elemento
                item_path = os.path.join(carpeta_caso, item)
                # Verificar si el elemento es un archivo y si tiene la extensión .png
                if os.path.isfile(item_path) and item.endswith('.png'):
                    # Obtener la letra de la carpeta de origen
                    letra_origen = carpeta_caso.split('/')[-1]
                    # Verificar si la letra de la carpeta de origen coincide con la letra de la columna
                    if letra_origen == columna.split('_')[0]:
                        # Copiar el archivo a la carpeta de destino
                        shutil.copy(item_path, ruta_destino)


# 1. `B_contempo`

## 1.1. Muestras de testeo

Ahora corresponde mover imágenes desde `train` hacia `test`. Tomaré el 20% de cada categoría, `0` y `1` en este caso.

Previamente he notado que habían <mark>imágenes corruptas</mark> (cuando intentaba abrirlas, el visor indicaba problemas. Eran aquellas que iniciaban su nombre con un punto. P. ej. "._B1.png"), las he tenido que sacar de la carpeta.


In [54]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

# Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

# Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 29
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 69
El 20% del número total de imágenes para la clase 1 es: 20


## 1.2. Elaboración del modelo

Y ahora pruebo con una Red Neuronal Convulacional (CNN).

In [55]:
import tensorflow as tf
from keras import layers,models
import os
import numpy as np
import cv2
import random

In [56]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/mimodelo.keras')

#model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/mimodelo.keras')

Epoch 1/80
3/3 [==============================] - 2s 160ms/step - loss: 5.2590 - accuracy: 0.6875
Epoch 2/80
3/3 [==============================] - 1s 198ms/step - loss: 2.2559 - accuracy: 0.6250
Epoch 3/80
3/3 [==============================] - 1s 166ms/step - loss: 1.6068 - accuracy: 0.5750
Epoch 4/80
3/3 [==============================] - 1s 153ms/step - loss: 1.0106 - accuracy: 0.6250
Epoch 5/80
3/3 [==============================] - 0s 139ms/step - loss: 0.8869 - accuracy: 0.5250
Epoch 6/80
3/3 [==============================] - 1s 163ms/step - loss: 0.8566 - accuracy: 0.7000
Epoch 7/80
3/3 [==============================] - 0s 148ms/step - loss: 0.6999 - accuracy: 0.7000
Epoch 8/80
3/3 [==============================] - 0s 137ms/step - loss: 0.6424 - accuracy: 0.6750
Epoch 9/80
3/3 [==============================] - 0s 92ms/step - loss: 0.6691 - accuracy: 0.6375
Epoch 10/80
3/3 [==============================] - 0s 93ms/step - loss: 0.6087 - accuracy: 0.7125
Epoch 11/80
3/3 [====

## 1.3. Pruebas con los datos de entrenamiento

Cuando pruebo el modelo con los datos de entrenamiento, sí predice bien.

In [57]:
ruta_predict = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/train/0/Screenshot 2024-05-08 at 12-57-06 Ático por 595.000 € de 104 metros calle de rufino blanco áticos en madrid en Fuente del Berro Madrid - habitaclia.png'

my_image = cv2.imread(ruta_predict)
my_image = cv2.resize(my_image, (width, height))

result = model.predict(np.array([my_image]))
print(result)

1/1 [==============================] - 0s 88ms/step
[[9.9999088e-01 2.2426262e-05]]


Pero cuando paso a los datos de testeo, ya es más variable. En estos primeros ejemplos, falla.

In [60]:
ruta_predict = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/0/Screenshot 2024-05-07 at 13-59-12 Piso por 146.500 € de 74 metros carrer de la sardana piso alquilado en venta en calle sardana sant andreu de la barca en Sant Andreu de la Barca - habitaclia.png'

my_image = cv2.imread(ruta_predict)
my_image = cv2.resize(my_image, (width, height))

result = model.predict(np.array([my_image]))
print(result)

1/1 [==============================] - 0s 27ms/step
[[0.4373248  0.77704704]]


In [61]:
ruta_predict = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/0/B3.png'

my_image = cv2.imread(ruta_predict)
my_image = cv2.resize(my_image, (width, height))

result = model.predict(np.array([my_image]))
print(result)

1/1 [==============================] - 0s 31ms/step
[[0.02163234 0.99166715]]


Y en este acierta. Podemos estar ante un caso de <mark>overtfitting</mark>.

In [62]:
ruta_predict = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/1/Screenshot 2024-05-07 at 15-15-54 Piso por 1.550.000 € de 139 metros piso de 3 dormitorios en el paseo de gràcia en el eixample derecho barcelona en Dreta de l´Eixample Barcelona - habitaclia.png'

my_image = cv2.imread(ruta_predict)
my_image = cv2.resize(my_image, (width, height))

result = model.predict(np.array([my_image]))
print(result)

1/1 [==============================] - 0s 23ms/step
[[0.00139691 0.9993208 ]]


## 1.4. Pruebas con los datos de testeo

Volveré a correr el modelo pero tomando los datos de la carpeta `test` para calcular el accuracy.

In [63]:
# Definir el ancho y alto de las imágenes
width = 240
height = 240

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 301ms/step - loss: 1.2869 - accuracy: 0.6667
Accuracy en los datos de prueba: 0.6666666865348816


# 2. `B_estado`

In [64]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = 20
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 9
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 34
El 20% del número total de imágenes para la clase 1 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 67
El 20% del número total de imágenes para la clase 2 es: 20


In [66]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 303ms/step - loss: 0.8539 - accuracy: 0.6000
Accuracy en los datos de prueba: 0.6000000238418579


# 3. `B_lavamano`

In [67]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 41
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 53
El 20% del número total de imágenes para la clase 1 es: 20


In [68]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/B_lavamano/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 293ms/step - loss: 0.7347 - accuracy: 0.5000
Accuracy en los datos de prueba: 0.5


# 4. `C_contempo`

In [69]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 38
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 57
El 20% del número total de imágenes para la clase 1 es: 20


In [71]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 443ms/step - loss: 1.4484 - accuracy: 0.6471
Accuracy en los datos de prueba: 0.6470588445663452


# 5. `C_estado`

In [72]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = 20
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 6
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 28
El 20% del número total de imágenes para la clase 1 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 61
El 20% del número total de imágenes para la clase 2 es: 20


In [74]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 273ms/step - loss: 1.0441 - accuracy: 0.4118
Accuracy en los datos de prueba: 0.4117647111415863


# 6. `C_armarios`

In [75]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)


Número total de imágenes en la carpeta de entrenamiento para la clase 0: 4
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 85
El 20% del número total de imágenes para la clase 1 es: 20


In [76]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/C_armarios/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 374ms/step - loss: 0.0075 - accuracy: 1.0000
Accuracy en los datos de prueba: 1.0


# 7. `R_contemporaneo`

In [77]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 68
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 71
El 20% del número total de imágenes para la clase 1 es: 20


In [78]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_contempo/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 458ms/step - loss: 1.2742 - accuracy: 0.6296
Accuracy en los datos de prueba: 0.6296296119689941


# 8. `R_estado`

In [79]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/test/1"

# Ruta de la carpeta de entrenamiento y prueba para la clase 2
train_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/train/2"
test_folder_2 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/test/2"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

### Repetir el proceso para la clase 2
num_images_train_2 = len(os.listdir(train_folder_2))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 2:", num_images_train_2)

# Calcular el 20% de esa cantidad
percentage_20_train_2 = 20
print("El 20% del número total de imágenes para la clase 2 es:", percentage_20_train_2)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 2
move_random_images(train_folder_2, test_folder_2, percentage_20_train_2)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 3
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 59
El 20% del número total de imágenes para la clase 1 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 2: 77
El 20% del número total de imágenes para la clase 2 es: 20


In [80]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(3),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_estado/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 280ms/step - loss: 1.0142 - accuracy: 0.5769
Accuracy en los datos de prueba: 0.5769230723381042


# 9. `R_singular`

In [81]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 113
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 26
El 20% del número total de imágenes para la clase 1 es: 20


In [83]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_singular/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 291ms/step - loss: 1.7829 - accuracy: 0.8148
Accuracy en los datos de prueba: 0.8148148059844971


# 10. `R_ventana`

In [84]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 80
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 56
El 20% del número total de imágenes para la clase 1 es: 20


In [85]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_ventana/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 432ms/step - loss: 1.5806 - accuracy: 0.6296
Accuracy en los datos de prueba: 0.6296296119689941


# 11. `R_carpinteria`

In [86]:
# Ruta de la carpeta de entrenamiento y prueba para la clase 0
train_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/train/0"
test_folder_0 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/test/0"

# Ruta de la carpeta de entrenamiento y prueba para la clase 1
train_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/train/1"
test_folder_1 = "/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/test/1"

# Función para mover imágenes aleatorias a la carpeta de prueba
def move_random_images(source_folder, destination_folder, percentage):
    # Obtener la lista de archivos en la carpeta de origen
    files = os.listdir(source_folder)
    # Calcular el número de imágenes que representan el porcentaje dado
    num_images_to_move = int(len(files) * percentage / 100)
    # Elegir imágenes al azar
    images_to_move = random.sample(files, num_images_to_move)
    # Mover las imágenes seleccionadas a la carpeta de destino
    for image in images_to_move:
        shutil.move(os.path.join(source_folder, image), destination_folder)

### Contar el número total de imágenes en la carpeta de entrenamiento para la clase 0
num_images_train_0 = len(os.listdir(train_folder_0))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 0:", num_images_train_0)

# Calcular el 10% de esa cantidad
percentage_20_train_0 = 20
print("El 20% del número total de imágenes para la clase 0 es:", percentage_20_train_0)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 0
move_random_images(train_folder_0, test_folder_0, percentage_20_train_0)

### Repetir el proceso para la clase 1
num_images_train_1 = len(os.listdir(train_folder_1))
print("Número total de imágenes en la carpeta de entrenamiento para la clase 1:", num_images_train_1)

# Calcular el 10% de esa cantidad
percentage_20_train_1 = 20
print("El 20% del número total de imágenes para la clase 1 es:", percentage_20_train_1)

# Mover imágenes aleatorias a la carpeta de prueba para la clase 1
move_random_images(train_folder_1, test_folder_1, percentage_20_train_1)

Número total de imágenes en la carpeta de entrenamiento para la clase 0: 25
El 20% del número total de imágenes para la clase 0 es: 20
Número total de imágenes en la carpeta de entrenamiento para la clase 1: 114
El 20% del número total de imágenes para la clase 1 es: 20


In [87]:
width = 240
height = 240

ruta_train = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/train/'

train_x = []
train_y = []

labels = os.listdir(ruta_train)

for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(2),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 80

model.fit(x_data, y_data, epochs = epochs, verbose=0)

models.save_model(model, '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/mimodelo.keras')

#####AQUI CALCULO EL ACCURACY CON LOS DATOS DE TESTEO

# Ruta de los datos de prueba
ruta_test = '/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/test/'

test_x = []
test_y = []

# Obtener la lista de etiquetas
labels = os.listdir(ruta_test)

# Iterar sobre las carpetas de las etiquetas
for i in os.listdir(ruta_test):
    # Iterar sobre las imágenes dentro de cada carpeta
    for j in os.listdir(ruta_test + i):
        # Leer la imagen y redimensionarla
        img = cv2.imread(ruta_test+i+'/'+j)
        resized_image = cv2.resize(img, (width,height))

        test_x.append(resized_image)

        # Crear la etiqueta correspondiente
        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                test_y.append(array)

# Convertir los datos a arrays numpy
x_test = np.array(test_x)
y_test = np.array(test_y)

# Cargar el modelo entrenado
model = models.load_model('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/orden_tercer_intento/R_carpinteria/mimodelo.keras')

# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy en los datos de prueba:", accuracy)

1/1 [==============================] - 0s 285ms/step - loss: 1.4781 - accuracy: 0.7407
Accuracy en los datos de prueba: 0.7407407164573669
